In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt

pd.set_option('max_colwidth', 1000)
pd.set_option('display.max_rows', 999)
plt.style.use('ggplot')

# thoughts:
# * we should add in out of vocab words
# * fix #ERROR
# * add scores for 3 threat ratings with no scores
# * include more thatn 200 words (or 200 from start 200 from end)

/Users/ericagreene/Envs/dawid/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## load data


In [103]:
# read hat scored data
base_path = '/Users/ericagreene/Projects/conversationai-models/kaggle-classification'
error_rates_path = base_path + '/error_rates_all_q42017.csv'
annoation_path = base_path + '/dawid_training.csv'
comments_path = base_path + '/q42017_unit_id_comment_text.csv'
predictions_path = base_path + '/predictions_all_q42017.csv'

df_acc = pd.read_csv(tf.gfile.Open(error_rates_path, 'r')) # per-worker accuracy rates
df_annotation = pd.read_csv(tf.gfile.Open(annoation_path, 'r')) # raw annotations 
df_comments = pd.read_csv(tf.gfile.Open(comments_path, 'r'))
df_predictions = pd.read_csv(tf.gfile.Open(predictions_path, 'r'))


In [131]:
# add mean scores to raw annotations
df_mean = df_annotation.groupby(by='_unit_id').mean().reset_index()\

df_1 = df_annotation.merge(df_mean, on=['_unit_id'], suffixes=(['', '_mean']))

print len(df_1)

# add comment text
df_2 = df_1.merge(df_comments, on='_unit_id') 

# add dawid-skene predictions
df = df_2.merge(df_predictions, on='_unit_id', suffixes=(['', '_hat']))

9368969


In [134]:
df.head()

,_unit_id,_worker_id,identity_hate,flirtation,insult,obscene,sexual_explicit,threat,toxic_score,_worker_id_mean,...,sexual_explicit_mean_hat,sexual_explicit_hat,threat_mean_hat,threat_hat,flirtation_mean_hat,flirtation_hat,toxic_score_mean_hat,toxic_score_hat,toxic_score_hat_frac_very_neg,toxic_score_hat_frac_neg
0,1520352507,41616896,0,0,0,0,0,0,0,42043797.8,...,0.0,0.0,0.0,0.0012,0.0,0.001,0.4,0.8249,0.0,0.0014
1,1520352507,37061905,0,0,0,0,0,0,0,42043797.8,...,0.0,0.0,0.0,0.0012,0.0,0.001,0.4,0.8249,0.0,0.0014
2,1520352507,44112385,0,0,0,0,0,0,0,42043797.8,...,0.0,0.0,0.0,0.0012,0.0,0.001,0.4,0.8249,0.0,0.0014
3,1520352507,40445979,0,0,0,0,0,0,0,42043797.8,...,0.0,0.0,0.0,0.0012,0.0,0.001,0.4,0.8249,0.0,0.0014
4,1520352507,44096613,0,0,0,0,0,0,0,42043797.8,...,0.0,0.0,0.0,0.0012,0.0,0.001,0.4,0.8249,0.0,0.0014


## what are the characteristics of low-accuracy annotators? 
1. are there people who are high accuracy for some categories and low accuracy for others? 
2. who are the high and low accuracy identity hate raters?  
3. who are the low accuracy for class 0 raters? 
4. examples of high accuracy raters who disagree with the consensus and are right 
5. examples of someone who often disagrees with average but is right 



In [140]:
# 5. examples of someone who often disagrees with average but is right 

label = 'obscene'
mean_label = label + '_mean'
mean_diff_label = label + '_mean_diff'


df[mean_diff_label] = np.abs(df[label] - df[mean_label])

cols = [label, mean_label, mean_diff_label, '_worker_id', 'comment_text']

df_diff = df.sort_values(by=mean_diff_label, ascending=False)[cols]

# look at examples where people are predicting 1, YES OBSCENE
df_diff[df_diff[label] == 1][0:100]


,obscene,obscene_mean,obscene_mean_diff,_worker_id,comment_text
3356916,1,0.066667,0.933333,40794553,""" “The terrorists are the Americans, and everyone knows it.""\n\nDon't you just love (sarc) how muslims scream what amounts to ""he hit me back!!!!""?"
6168149,1,0.066667,0.933333,43650017,It going to be hard for these politicians and pressitutes to work when their pimp gets arrested.
5943399,1,0.066667,0.933333,43608691,It would be better if it was Husseinus Theanus......
860546,1,0.071429,0.928571,43650017,"Yep, and here's Bomber with his first secret message of the day. When are you going to send me that secret decoder ring, Bomber? I'll bet that what you said was a real hoot. Not just a 10 on the laugh meter but clear off the scale. Is it the 'secret' story of how all of your 'ancestors' died in Auschwitz?"
5471413,1,0.071429,0.928571,44043679,"What's that beautiful line by Tim Wise? Something like: ""Do I expect you to feel guilty? No, I expect you to be angry."""
2412118,1,0.071429,0.928571,40794553,"""If these jihadis don't want to be killed or imprisoned, I suggest they don't join the war against America in this first place.""\nBut again, you assume they are jihadis because they have found their way into Guantanamo. That is not neccessarily the case. What do you do if you are not a jihadi, but get mistake for one? There is basically nothing you are able to do. \n\nBoth WW2 examples you use are flawed. PoW camps did not require a trial because they did not stand accused of anything. There decision to fight was a lot more passive than the decision to join a terrorist insurgency, as it was by and large compulsory for all males of a certain age of that time. They were not seen as criminals, hence why not every German soldier was put on trial. But joining a terrorist group is quite rightly criminal activity. But if you stand accused of criminal activity, you have the right to a fair trial. \nAs for Nuremberg, it was not as if they lacked the evidence to bring those people to trial d..."
4650702,1,0.071429,0.928571,39297753,100% ^^^^
6521114,1,0.076923,0.923077,42816524,"I would say that sums up much of it!\n\n\nWe survived the likes of R.Hill, T_Bone and Mabus I don't see any reason not to keep kicking some butt in the new year my friend!"
3101790,1,0.076923,0.923077,39297753,"Excellent suggestion. OTOH, if Perry's people are prepared to not just catch Obama's people doing un-American things, but to get those facts to the media in real time, this could be good. It might also get a whole lot of back-off from Obama's minions. The National Guard could basically do recon on the traitorous DHS and the Obamafied military."
2003278,1,0.076923,0.923077,44016475,Terrorists should go to jail
